In [1]:
%pip install pandas numpy glob fuzzywuzzy

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
ERROR: No matching distribution found for glob
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import glob as glob
from fuzzywuzzy import process

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
def process_company_file(file_path, combined_df=None):
    """
    Process a single company CSV file and append it to an existing DataFrame.

    Args:
        file_path (str): Path to the company's CSV file.
        combined_df (pd.DataFrame): Existing DataFrame to append to. Default is None.

    Returns:
        pd.DataFrame: Updated combined DataFrame with the processed company's data.
    """
    try:
        # Load the CSV file, skipping unnecessary rows
        df = pd.read_csv(file_path, encoding='latin1', skiprows=4).dropna()

        # Remove rows containing "Growth (YoY)" in the first column
        df = df[~df.iloc[:, 0].str.contains('Growth', na=False)]

        # Drop unnamed column if it exists
        if 'Unnamed: 1' in df.columns:
            df = df.drop(columns=['Unnamed: 1'])

        # Transpose the DataFrame
        df = df.T

        # Reset index to clean up transposed DataFrame
        df = df.reset_index()

        # Set the first row as the column headers
        df.columns = df.iloc[0]
        df = df.drop(0)

        # Rename the "12 Months Ending" column to "Year"
        if '12 Months Ending' in df.columns:
            df = df.rename(columns={'12 Months Ending': 'Year'})

        # Extract year from date strings (e.g., "12/31/2014" → 2014)
        if 'Year' in df.columns:
            df['Year'] = df['Year'].str.extract(r'(\d{4})').astype(int)

        # Fill missing values with NaN
        df = df.replace('', np.nan)

        # Reload original file to extract company and ticker info
        raw_df = pd.read_csv(file_path, encoding='latin1')

        # Extract company name and ticker from the first row of the raw file
        company, ticker = raw_df.iloc[0, 0].split('(')
        company = company.strip()
        ticker = ticker.split(' ')[0].replace('/', '.')

        # Add company and ticker columns to DataFrame
        df['Company'] = company
        df['Ticker'] = ticker

        # Set index as Ticker and Year for better organization
        df = df.set_index(['Company', 'Ticker', 'Year'])

        # Append to combined DataFrame
        if combined_df is None:
            combined_df = pd.DataFrame()  # Initialize if not provided

        combined_df = pd.concat([combined_df, df])

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

    return combined_df

In [4]:
# Initialize an empty DataFrame for all companies
all_companies_df = pd.DataFrame()

# Path where all company files are stored (adjust as needed)
folder_path = 'Energy/*.csv'

# Process each file and append data to the combined DataFrame
for file in glob.glob(folder_path):
    all_companies_df = process_company_file(file, combined_df=all_companies_df)

# Reset index for final DataFrame (optional)
all_companies_df.reset_index(inplace=True)

# Save the final combined DataFrame to a CSV file (optional)
all_companies_df.to_csv('combined_energy_data.csv', index=False)

/var/folders/08/xxsq81ks0gx_hgrmn3w7_0p00000gn/T/ipykernel_39640/761541624.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('', np.nan)
/var/folders/08/xxsq81ks0gx_hgrmn3w7_0p00000gn/T/ipykernel_39640/761541624.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('', np.nan)
/var/folders/08/xxsq81ks0gx_hgrmn3w7_0p00000gn/T/ipykernel_39640/761541624.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

Error processing Energy/Barnwell Industries.csv: Reindexing only valid with uniquely valued Index objects


/var/folders/08/xxsq81ks0gx_hgrmn3w7_0p00000gn/T/ipykernel_39640/761541624.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('', np.nan)
/var/folders/08/xxsq81ks0gx_hgrmn3w7_0p00000gn/T/ipykernel_39640/761541624.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('', np.nan)
/var/folders/08/xxsq81ks0gx_hgrmn3w7_0p00000gn/T/ipykernel_39640/761541624.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicit

Error processing Energy/PTT.csv: Error tokenizing data. C error: Expected 12 fields in line 28, saw 13

Error processing Energy/1. Energy Company Market Cap.csv: not enough values to unpack (expected 2, got 1)


/var/folders/08/xxsq81ks0gx_hgrmn3w7_0p00000gn/T/ipykernel_39640/761541624.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('', np.nan)
/var/folders/08/xxsq81ks0gx_hgrmn3w7_0p00000gn/T/ipykernel_39640/761541624.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('', np.nan)
/var/folders/08/xxsq81ks0gx_hgrmn3w7_0p00000gn/T/ipykernel_39640/761541624.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

In [5]:
# Load market cap data
market_cap_data = pd.read_csv('Energy/1. Energy Company Market Cap.csv', skiprows=7)

market_cap_data = market_cap_data[['Company', 'Ticker', 'Market Cap ($M)']]

# Clean and convert Market Cap to numeric
market_cap_data['Market Cap ($M)'] = pd.to_numeric(market_cap_data['Market Cap ($M)'], errors='coerce')

# Extract ticker
market_cap_data['Ticker'] = market_cap_data['Ticker'].str.split(' ').str[0].str.replace('/', '.')

market_cap_data.head()

market_cap_data.to_csv('energy_market_cap_data_cleaned.csv', index=False)

In [6]:
def calculate_percentiles(df):
    """
    Calculate the 95th, 75th, and 25th percentiles of market capitalization.
    """
    percentiles = df['Market Cap ($M)'].quantile([0.95, 0.75, 0.25])
    return percentiles[0.95], percentiles[0.75], percentiles[0.25]

def classify_market_cap(market_cap, p95, p75, p25):
    """
    Classify companies based on market capitalization percentiles.
    """
    if pd.isna(market_cap):
        return 'Unknown'
    elif market_cap < p25:
        return 'Small Cap'
    elif p25 <= market_cap < p75:
        return 'Mid Cap'
    elif p75 <= market_cap < p95:
        return 'Large Cap'
    else:
        return 'Mega Cap'

p95, p75, p25 = calculate_percentiles(market_cap_data)

# Apply classification to the Market Cap column
market_cap_data['Market Cap Category'] = market_cap_data['Market Cap ($M)'].apply(lambda x: classify_market_cap(x, p95, p75, p25))

In [7]:
# Merge by Ticker first (direct match)
all_companies_df = pd.merge(
    all_companies_df,
    market_cap_data[['Ticker', 'Market Cap ($M)', 'Market Cap Category']],
    on='Ticker',
    how='left'
)

all_companies_df.head()

,Company,Ticker,Year,BESG ESG Score,BESG Environmental Pillar Score,BESG Social Pillar Score,BESG Governance Pillar Score,ESG Disclosure Score,Environmental Disclosure Score,Social Disclosure Score,...,Indep Directors,Raw Materials Used,Percentage Gender Pay Gap Employees Ex Management,Percentage Raw Material from Sustainable Sources,"Revenue, Adj","Net Income, Adj",Margin %,"EPS, Adj",Market Cap ($M),Market Cap Category
0,Peyto Exploration & Development Corp,PEY,2014,NaN,NaN,NaN,NaN,38.879,12.896,18.047,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2321.1,Mid Cap
1,Peyto Exploration & Development Corp,PEY,2015,2.940,2.180,1.580,6.300,41.092,14.316,22.703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2321.1,Mid Cap
2,Peyto Exploration & Development Corp,PEY,2016,3.160,2.770,1.580,5.930,44.050,20.054,25.846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2321.1,Mid Cap
3,Peyto Exploration & Development Corp,PEY,2017,2.670,1.860,1.550,5.790,45.478,21.776,28.416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2321.1,Mid Cap
4,Peyto Exploration & Development Corp,PEY,2018,2.800,1.850,1.550,6.510,44.825,22.199,28.416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2321.1,Mid Cap


In [8]:
print(all_companies_df.shape)
# Summary of classifications
all_companies_df['Market Cap Category'].value_counts()

(1234, 141)


Market Cap Category
Mid Cap      475
Large Cap    234
Small Cap    188
Unknown       50
Mega Cap      50
Name: count, dtype: int64

In [9]:
# List of companies with missing market cap data
missing_market_cap = all_companies_df[all_companies_df['Market Cap ($M)'].isnull()]['Company'].unique()

missing_market_cap

array(['Trecora LLC', 'Mesquite Energy Inc', 'Vivo Energy Ltd',
       'Floene Energias SA', 'Bumi Armada Bhd', 'Braskem SA',
       'Italgas SpA', 'Petroleos Mexicanos', 'Navigator Holdings Ltd',
       'Ecopetrol SA', 'LG Chem Ltd',
       'CorEnergy Infrastructure Trust Inc',
       'Lotte Chemical Titan Holding Bhd',
       'China National Petroleum Corp', 'Kerry Group PLC',
       'Saudi Arabian Oil Co', 'ITC Holdings Corp', 'PDC Energy Inc',
       'Arabian Drilling Co', 'Sapura Energy Bhd', 'Lotte Chemical Corp',
       'Petronas Gas Bhd', 'Brookfield Corp',
       'Sahara International Petrochemical Co',
       'Seven Generations Energy Ltd', 'Fonterra Co-operative Group Ltd',
       'Buckeye Partners LP', 'Posco International Corp',
       'Senex Energy Pty Ltd', 'Kansai Electric Power Co Inc/The'],
      dtype=object)

In [10]:
def fuzzy_match_company(company_name, market_cap_data):
    """
    Perform fuzzy matching for a company name against the market cap data.

    Args:
        company_name (str): The name of the company to match.
        market_cap_data (pd.DataFrame): The DataFrame containing market cap information.

    Returns:
        tuple: Market Cap ($M) and Market Cap Category if a match is found; otherwise (None, None).
    """
    # Clean up company names for better matching
    suffixes = ['Ltd.', 'Inc', 'Corp', 'Plc', 'Bhd', 'Pty Ltd', 'Co', 'SA', 'NV']
    for suffix in suffixes:
        company_name = company_name.replace(suffix, '')
    company_name = company_name.strip()

    # Clean up market_cap_data['Company'] for consistent matching
    market_cap_data['Company'] = market_cap_data['Company'].str.split('(').str[0].replace(
        {'Ltd.': '',
            'Inc.': '',
            'Corp.': '',
            'P.l.c.': '',
            'Berhad': '',
            'Pty Ltd': '',
            'Co.': '',
            'S.A.': '',
            'N.V.': '',
            'Limited': ''
         }, regex=True).str.strip()

    # Perform fuzzy matching
    match = process.extractOne(company_name, market_cap_data['Company'])

    # If a high-confidence match is found, return Market Cap and Category
    if match and match[1] > 90:
        matched_row = market_cap_data[market_cap_data['Company'] == match[0]].iloc[0]
        print(f'Matched: {company_name} -> {matched_row["Company"]} (Score: {match[1]})')
        return matched_row['Market Cap ($M)'], matched_row['Market Cap Category']

    elif company_name.startswith('PTT'):
        print(f'Matched: {company_name} -> PTT Exploration and Production Public Company Limited')
        matched_row = market_cap_data[market_cap_data['Ticker'] == 'PTTEP']
        market_cap = matched_row['Market Cap ($M)'].values[0]
        category = matched_row['Market Cap Category'].values[0]
        return market_cap, category

    elif company_name.startswith('LG Chem'):
        print(f'Matched: {company_name} -> LG Chem, Ltd.')
        matched_row = market_cap_data[market_cap_data['Ticker'] == 'A051910']
        market_cap = matched_row['Market Cap ($M)'].values[0]
        category = matched_row['Market Cap Category'].values[0]
        return market_cap, category

    elif company_name.startswith('Lotte'):
        print(f'Matched: {company_name} -> Lotte Chemical Corporation')
        matched_row = market_cap_data[market_cap_data['Ticker'] == 'A011170']
        market_cap = matched_row['Market Cap ($M)'].values[0]
        category = matched_row['Market Cap Category'].values[0]
        return market_cap, category

    return None, None


In [11]:
# Iterate over missing companies and attempt to match
for company in missing_market_cap:
    matched_market_cap, matched_category = fuzzy_match_company(company, market_cap_data)

    # Update all_companies_df if a match is found
    if matched_market_cap is not None and matched_category is not None:
        mask = all_companies_df['Company'] == company
        if mask.any():  # Check if any rows match the condition
            print(f"Updating {company} with Market Cap: {matched_market_cap}, Category: {matched_category}")
            all_companies_df.loc[mask, 'Market Cap ($M)'] = matched_market_cap
            all_companies_df.loc[mask, 'Market Cap Category'] = matched_category
        else:
            print(f"Warning: Match found for {company}, but no rows updated.")

Matched: Mesquite Energy -> Mesquite Energy, (Score: 100)
Updating Mesquite Energy Inc with Market Cap: nan, Category: Unknown
Matched: Vivo Energy Ltd -> Vivo Energy (Score: 95)
Updating Vivo Energy Ltd with Market Cap: nan, Category: Unknown
Matched: Floene Energias -> Floene Energias, (Score: 100)
Updating Floene Energias SA with Market Cap: nan, Category: Unknown
Matched: Bumi Armada -> Bumi Armada (Score: 100)
Updating Bumi Armada Bhd with Market Cap: 771.4, Category: Small Cap
Matched: Braskem -> Braskem (Score: 100)
Updating Braskem SA with Market Cap: 1794.0, Category: Small Cap
Matched: Petroleos Mexicanos -> Petróleos Mexicanos (Score: 97)
Updating Petroleos Mexicanos with Market Cap: nan, Category: Unknown
Matched: Ecopetrol -> Ecopetrol (Score: 100)
Updating Ecopetrol SA with Market Cap: 21625.0, Category: Mid Cap
Matched: LG Chem Ltd -> LG Chem, Ltd.
Updating LG Chem Ltd with Market Cap: 13058.8, Category: Mid Cap
Matched: rEnergy Infrastructure Trust -> Energy Infrastruct

In [12]:
updated_missing_market_cap = all_companies_df[all_companies_df['Market Cap ($M)'].isnull()]['Company'].unique()

updated_missing_market_cap

array(['Trecora LLC', 'Mesquite Energy Inc', 'Vivo Energy Ltd',
       'Floene Energias SA', 'Italgas SpA', 'Petroleos Mexicanos',
       'Navigator Holdings Ltd', 'China National Petroleum Corp',
       'ITC Holdings Corp', 'PDC Energy Inc', 'Brookfield Corp',
       'Seven Generations Energy Ltd', 'Buckeye Partners LP',
       'Senex Energy Pty Ltd', 'Kansai Electric Power Co Inc/The'],
      dtype=object)

In [13]:
all_companies_df['Market Cap Category'].value_counts()

Market Cap Category
Mid Cap      554
Small Cap    256
Large Cap    234
Unknown       74
Mega Cap      56
Name: count, dtype: int64

In [14]:
# Fill remaining missing values with Unknown
all_companies_df['Market Cap Category'] = all_companies_df['Market Cap Category'].fillna('Unknown')

print(all_companies_df['Market Cap Category'].isna().sum())

all_companies_df['Market Cap Category'].value_counts()

0


Market Cap Category
Mid Cap      554
Small Cap    256
Large Cap    234
Unknown      134
Mega Cap      56
Name: count, dtype: int64

In [15]:
all_companies_df.drop_duplicates(subset=['Company', 'Ticker', 'Year'], inplace=True)

all_companies_df['Market Cap Category'].value_counts()

Market Cap Category
Mid Cap      554
Small Cap    247
Large Cap    233
Unknown      134
Mega Cap      56
Name: count, dtype: int64

In [16]:
all_companies_df = all_companies_df.sort_values(by=['Company', 'Year'])

# Reindex
all_companies_df.reset_index(drop=True, inplace=True)

# Clean Margin % column name
all_companies_df.rename(columns={'  Margin %': 'Margin %'}, inplace=True)

all_companies_df.to_csv('energy_cleaned.csv', index=False)

In [17]:
all_companies_df.head()

,Company,Ticker,Year,BESG ESG Score,BESG Environmental Pillar Score,BESG Social Pillar Score,BESG Governance Pillar Score,ESG Disclosure Score,Environmental Disclosure Score,Social Disclosure Score,...,Indep Directors,Raw Materials Used,Percentage Gender Pay Gap Employees Ex Management,Percentage Raw Material from Sustainable Sources,"Revenue, Adj","Net Income, Adj",Margin %,"EPS, Adj",Market Cap ($M),Market Cap Category
0,ACS Actividades de Construccion y Servicios SA,ACS,2014,NaN,NaN,NaN,NaN,50.729,37.511,33.676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13716.9,Mid Cap
1,ACS Actividades de Construccion y Servicios SA,ACS,2015,1.850,0.660,2.380,3.660,55.296,42.616,41.717,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13716.9,Mid Cap
2,ACS Actividades de Construccion y Servicios SA,ACS,2016,2.170,1.130,2.370,4.050,55.497,48.354,36.578,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13716.9,Mid Cap
3,ACS Actividades de Construccion y Servicios SA,ACS,2017,2.760,1.130,4.300,4.110,57.248,50.317,44.891,...,NaN,"5,140.650",NaN,NaN,NaN,NaN,NaN,NaN,13716.9,Mid Cap
4,ACS Actividades de Construccion y Servicios SA,ACS,2018,2.630,1.110,4.200,3.640,62.609,52.280,59.039,...,NaN,"8,653.300",NaN,NaN,NaN,NaN,NaN,NaN,13716.9,Mid Cap
